In [251]:
##Import
import glob, os
from Bio import SeqIO
import pandas as pd
from collections import Counter
import fuzzysearch
import matplotlib as plt
import numpy as np
import itertools
plt.rcParams['pdf.fonttype'] = 42

In [252]:
##Load key of conditions and files
samples = pd.read_excel("recode.xls")

In [21]:
samples

,Phage,Gene,Plasmid,Direction,edit name 1,edit name 2,edit name 3,edit name 4,edit name 5,edit name 6,...,L_inside_10,R_inside_10,L_inside_11,R_inside_11,L_outside,R_outside,Rep1,Rep2,Rep3,Rep4
0,Lambda,L,pCF.88,R,NaN,NaN,NaN,NaN,T37C,C46T,...,NaN,NaN,NaN,NaN,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-20,msSBK-32-31,msCF-2-07,msCF-2-20
1,Lambda,L,pCF.89,R,NaN,NaN,NaN,C28T,T37C,C46T,...,NaN,NaN,NaN,NaN,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-21,msSBK-32-32,NaN,msCF-2-21
2,Lambda,L,pCF.90,R,NaN,NaN,C19T,C28T,T37C,C46T,...,NaN,NaN,NaN,NaN,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-22,msSBK-32-33,msCF-2-09,msCF-2-22
3,Lambda,L,pCF.91,R,NaN,G10A,C19T,C28T,T37C,C46T,...,GNTTCCTTTC,ATTAANAAAC,NaN,NaN,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-23,msSBK-32-34,msCF-2-10,msCF-2-23
4,Lambda,L,pCF.92,R,C4T,G10A,C19T,C28T,T37C,C46T,...,GNTTCCTTTC,ATTAANAAAC,TTTCNATTAA,AAACTTTCGC,TATGACCAGCCAACGTCCGA,ACTTTCGCAGTAAATCCCAT,msSBK-32-24,msSBK-32-35,NaN,msCF-2-24


In [10]:
ls

all_plasmids_double_edit_output.xlsx   msSBK-32-21_S21_L001_R1_001.fastq.gz
double_Edit_site_analysis-Copy1.ipynb  msSBK-32-22_S22_L001_R1_001.fastq.gz
double_edit_fastqs/                    msSBK-32-23_S23_L001_R1_001.fastq.gz
double_edit_placement.xls              msSBK-32-24_S24_L001_R1_001.fastq.gz
double_edits.pzfx                      msSBK-32-31_S31_L001_R1_001.fastq.gz
msCF-2-07_S7_L001_R1_001.fastq.gz      msSBK-32-32_S32_L001_R1_001.fastq.gz
msCF-2-09_S9_L001_R1_001.fastq.gz      msSBK-32-33_S33_L001_R1_001.fastq.gz
msCF-2-10_S10_L001_R1_001.fastq.gz     msSBK-32-34_S34_L001_R1_001.fastq.gz
msCF-2-20_S20_L001_R1_001.fastq.gz     msSBK-32-35_S35_L001_R1_001.fastq.gz
msCF-2-21_S21_L001_R1_001.fastq.gz     recode fastqs/
msCF-2-22_S22_L001_R1_001.fastq.gz     recode.xls
msCF-2-23_S23_L001_R1_001.fastq.gz     recode_Edit_site_analysis.ipynb
msCF-2-24_S24_L001_R1_001.fastq.gz     test.png
msSBK-32-20_S20_L001_R1_001.fastq.gz


In [11]:
#trimming reads with sickle
print ("="*60)
for file in glob.glob("ms*-*-*"):
    fn = file.split("_S")[0]
    print (fn)

    print ("\ntrimming files\n")
    cmd = "sickle se -f {0} -t sanger \
            -o {1}_trimmed_R1.fastq".format(file, fn)
    os.system(cmd)    
    print ("="*60)


msSBK-32-32

trimming files


FastQ records kept: 27019
FastQ records discarded: 2

msSBK-32-35

trimming files


FastQ records kept: 36557
FastQ records discarded: 5

msSBK-32-20

trimming files


FastQ records kept: 30715
FastQ records discarded: 1

msSBK-32-21

trimming files


FastQ records kept: 32810
FastQ records discarded: 2

msSBK-32-34

trimming files


FastQ records kept: 29509
FastQ records discarded: 0

msSBK-32-33

trimming files


FastQ records kept: 30386
FastQ records discarded: 0

msSBK-32-24

trimming files


FastQ records kept: 40309
FastQ records discarded: 1

msSBK-32-31

trimming files


FastQ records kept: 24948
FastQ records discarded: 2

msSBK-32-23

trimming files


FastQ records kept: 34129
FastQ records discarded: 1

msCF-2-07

trimming files


FastQ records kept: 36090
FastQ records discarded: 4

msSBK-32-22

trimming files


FastQ records kept: 35048
FastQ records discarded: 2

msCF-2-22

trimming files


FastQ records kept: 29460
FastQ records discarded:

In [22]:
##Globals
outcomes_dict = {}

In [23]:
#setting up df with information to pull out of samples df
n_edits = 11
c = ["fastq", "read", "frequency", "plasmid", "replicate"]
eds = ["edit nt {0}".format(i+1) for i in range(n_edits)]
c.extend(eds)
#print (eds)
df_reads = pd.DataFrame(columns=c)
display(df_reads)


,fastq,read,frequency,plasmid,replicate,edit nt 1,edit nt 2,edit nt 3,edit nt 4,edit nt 5,edit nt 6,edit nt 7,edit nt 8,edit nt 9,edit nt 10,edit nt 11


In [38]:
# modified seth fx to locate edit site and report nucleotide at that site 
##Defs
def extract_and_match(read, L_out, R_out, L_in, R_in):
    left_outside = fuzzysearch.find_near_matches(L_out,read,max_l_dist=2)
    right_outside = fuzzysearch.find_near_matches(R_out,read,max_l_dist=2)
    if len(left_outside) == 1 and len(right_outside) == 1:
        left_inside = fuzzysearch.find_near_matches(L_in,read,max_l_dist=1)
        right_inside = fuzzysearch.find_near_matches(R_in,read,max_l_dist=1)
        if len(left_inside) == 1 and len(right_inside) == 1:
            #print (left_inside, right_inside)
            var_nt = read[left_inside[0].end:right_inside[0].start]
            #print (var_nt)
            return (var_nt)
        else:
            return ("no_inside_flanking")
    else:
        return ("no_outside_flanking")




for i in samples.index:
    sample_i = int(i)
    outcomes_dict[i] = {}
    for rep in ["Rep1","Rep2","Rep3","Rep4"]:
        print (rep)
        fq = samples[rep].iloc[i]
        plasmid = samples["Plasmid"].iloc[i]
        #print (fq, "potato") 
        
        outcomes_dict[fq]= {"no_inside_flanking":0,"no_outside_flanking":0}
        all_reads_str = []
        read_counter = []
        
        try:
        # files are ~ msSBK-33-28_trimmed_R1.fastq
            fastq_reads = glob.glob("{0}_trimmed_R1.fastq".format(fq))[0]
            print (fastq_reads)

            for seq_record in SeqIO.parse(fastq_reads, "fastq"):
                all_reads_str.append(str(seq_record.seq))
            read_counter = Counter(all_reads_str)
            
            for read in read_counter:
                
                mutation_list = []
                
                ## brand new stuff here
                
                for j in range(n_edits): # for each mutation position: 
                    mut_pos = j+1
                    # mut_pos is the position of the mutation
                    L_out = samples["L_outside"].iloc[i]
                    R_out = samples["R_outside"].iloc[i]
                    L_in = samples["L_inside_{0}".format(mut_pos)].iloc[i]
                    R_in = samples["R_inside_{0}".format(mut_pos)].iloc[i]
                    
                    # print(L_in)
                    if L_in is np.nan:
                        mutation_list.append(np.nan)
                        continue 
                        
                    
                    # we want fx to return nucleotide!
                    
                
                    nucleotide = extract_and_match(read, L_out, R_out, L_in, R_in)
                    #print (nucleotide)

                    if nucleotide == "no_outside_flanking":
                        outcomes_dict[fq]["no_outside_flanking"] += read_counter[read] 
                        mutation_list.append(np.nan)

                    elif nucleotide == "no_inside_flanking":
                        outcomes_dict[fq]["no_inside_flanking"] += read_counter[read]
                        mutation_list.append(np.nan)

                    elif nucleotide in ["A", "T", "C", "G"]:
                        mutation_list.append(nucleotide)

                
                #print (mutation_list)
                
                # ["c", "c"]
                
                if len(mutation_list) == n_edits:
                    row_ = pd.DataFrame([[fq, str(read), int(read_counter[read]), plasmid, rep]+mutation_list],
                                    columns=['fastq', 'read','frequency', "plasmid", "replicate"]+\
                                        [' '.join(i) for i in list(zip(['edit nt']*len(mutation_list), [str(j) for j in range(1, len(mutation_list)+1)]))])
                                        # [' '.join(i) for i in list(zip(['edit nt']*len(mutation_list), range(1, len(mutation_list)+1)))])
                
                    #print (row_)
                    df_reads = pd.concat([df_reads, row_])
                    
                
                #display(df_reads)
            print (outcomes_dict[fq])
                             
                    
            #outcomes_dict[sample_i][rep][extract_and_match(read,i,rep)] += read_counter[read]
            #print(samples[rep][i])
        except IOError: #this happens when a file is missing
            print("%s missing" % samples[rep][i])
            
        except IndexError:
            print ("missing file {0}".format(fq))

Rep1
msSBK-32-20_trimmed_R1.fastq
{'no_inside_flanking': 764, 'no_outside_flanking': 41964}
Rep2
msSBK-32-31_trimmed_R1.fastq
{'no_inside_flanking': 751, 'no_outside_flanking': 31047}
Rep3
msCF-2-07_trimmed_R1.fastq
{'no_inside_flanking': 776, 'no_outside_flanking': 5121}
Rep4
msCF-2-20_trimmed_R1.fastq
{'no_inside_flanking': 492, 'no_outside_flanking': 819}
Rep1
msSBK-32-21_trimmed_R1.fastq
{'no_inside_flanking': 1310, 'no_outside_flanking': 74675}
Rep2
msSBK-32-32_trimmed_R1.fastq
{'no_inside_flanking': 929, 'no_outside_flanking': 65480}
Rep3
missing file nan
Rep4
msCF-2-21_trimmed_R1.fastq
{'no_inside_flanking': 945, 'no_outside_flanking': 12000}
Rep1
msSBK-32-22_trimmed_R1.fastq
{'no_inside_flanking': 1449, 'no_outside_flanking': 131040}
Rep2
msSBK-32-33_trimmed_R1.fastq
{'no_inside_flanking': 1433, 'no_outside_flanking': 102053}
Rep3
msCF-2-09_trimmed_R1.fastq
{'no_inside_flanking': 895, 'no_outside_flanking': 4487}
Rep4
msCF-2-22_trimmed_R1.fastq
{'no_inside_flanking': 1336, 'no_

In [39]:
#df_reads.to_excel("recode_after_function.xlsx")

In [45]:
# import json

# json = json.dumps(outcomes_dict)
# f = open("two_edits_outcomes_dict.json", "w")
# f.write(json)
# f.close()

In [40]:
# print(len(df_reads))
# display(df_reads)

136416


,fastq,read,frequency,plasmid,replicate,edit nt 1,edit nt 2,edit nt 3,edit nt 4,edit nt 5,edit nt 6,edit nt 7,edit nt 8,edit nt 9,edit nt 10,edit nt 11
0,msSBK-32-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,7374,pCF.92,Rep1,C,G,C,C,T,C,C,C,C,C,C
0,msSBK-32-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,3954,pCF.92,Rep1,C,G,C,C,T,C,C,C,C,C,C
0,msSBK-32-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,3,pCF.92,Rep1,C,G,C,C,T,C,C,C,C,C,C
0,msSBK-32-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,1,pCF.92,Rep1,C,G,C,T,C,T,T,T,T,C,C
0,msSBK-32-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,23,pCF.92,Rep1,C,G,C,C,T,C,C,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,msCF-2-24,GGCTGTCGCGGCTGAATATGACCAGCCAACGTCCGATATCACGAAG...,1,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
0,msCF-2-24,CGGCTGTCGCGGATGAATATGACAAGCA,1,pCF.92,Rep4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,msCF-2-24,CGGCTGTCGCGTGAATATCTCCAGATCGGAAGAGCACACGTCTGAA...,1,pCF.92,Rep4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,1,pCF.92,Rep4,C,G,NaN,NaN,NaN,C,C,C,NaN,NaN,C


In [48]:
#df_reads.sort_values(by=['frequency'], ascending=False).head(n=10)

,fastq,read,frequency,plasmid,replicate,edit nt 1,edit nt 2,edit nt 3,edit nt 4,edit nt 5,edit nt 6,edit nt 7,edit nt 8,edit nt 9,edit nt 10,edit nt 11
0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,24513,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,24513,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
0,msCF-2-10,AGGATGCTGCAGATCTGTTCCCTACATCACCGCCAATTGCAGATCG...,21106,pCF.91,Rep3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,msCF-2-21,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,21074,pCF.89,Rep4,NaN,NaN,NaN,C,T,C,C,C,NaN,NaN,NaN
0,msCF-2-10,AGGATGCTGCAGATCGGAAGAGTTTACATCACCGCCAATTGCAGAT...,20089,pCF.91,Rep3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,msCF-2-07,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,19586,pCF.88,Rep3,NaN,NaN,NaN,NaN,T,C,C,NaN,NaN,NaN,NaN
0,msCF-2-22,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,19498,pCF.90,Rep4,NaN,NaN,C,C,T,C,C,C,C,NaN,NaN
0,msCF-2-23,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,18701,pCF.91,Rep4,NaN,G,C,C,T,C,C,C,C,C,NaN
0,msCF-2-10,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,17790,pCF.91,Rep3,NaN,G,C,C,T,C,C,C,C,C,NaN
0,msCF-2-09,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,14642,pCF.90,Rep3,NaN,NaN,C,C,T,C,C,C,C,NaN,NaN


In [96]:
#from samples df, make dictionary of which edits are populated for each plasmid (some plasmids dont' have edit nt 1, 2, 3, etc.) 
plasmid_to_numedits = dict()
def find_numedits(plasmid):
    plasmid_edit_series = list(samples[samples['Plasmid']==plasmid][[''.join(j) for j in list(zip(['edit name ']*11, [str(i) for i in range(1, 12)]))]].dropna(axis=1).columns)
    plasmid_edit_series = ['edit nt ' + i.split(' ')[-1] for i in plasmid_edit_series]
    plasmid_to_numedits[plasmid] = plasmid_edit_series

samples['Plasmid'].apply(lambda x: find_numedits(x))

plasmid_to_numedits

{'pCF.88': ['edit nt 5', 'edit nt 6', 'edit nt 7'],
 'pCF.89': ['edit nt 4', 'edit nt 5', 'edit nt 6', 'edit nt 7', 'edit nt 8'],
 'pCF.90': ['edit nt 3',
  'edit nt 4',
  'edit nt 5',
  'edit nt 6',
  'edit nt 7',
  'edit nt 8',
  'edit nt 9'],
 'pCF.91': ['edit nt 2',
  'edit nt 3',
  'edit nt 4',
  'edit nt 5',
  'edit nt 6',
  'edit nt 7',
  'edit nt 8',
  'edit nt 9',
  'edit nt 10'],
 'pCF.92': ['edit nt 1',
  'edit nt 2',
  'edit nt 3',
  'edit nt 4',
  'edit nt 5',
  'edit nt 6',
  'edit nt 7',
  'edit nt 8',
  'edit nt 9',
  'edit nt 10',
  'edit nt 11']}

In [75]:
df_reads.reset_index().groupby("plasmid")

,index,fastq,read,frequency,replicate,edit nt 1,edit nt 2,edit nt 3,edit nt 4,edit nt 5,edit nt 6,edit nt 7,edit nt 8,edit nt 9,edit nt 10,edit nt 11
plasmid,,,,,,,,,,,,,,,,
pCF.88,20552,20552,20552,20552,20552,0,0,0,0,9426,9325,9238,0,0,0,0
pCF.89,22824,22824,22824,22824,22824,0,0,0,5433,5461,5425,5349,5388,0,0,0
pCF.90,20351,20351,20351,20351,20351,0,0,8571,8583,8598,8467,8387,8508,8661,0,0
pCF.91,45614,45614,45614,45614,45614,0,7356,7355,7340,7370,7278,7197,7249,7399,7158,0
pCF.92,27075,27075,27075,27075,27075,14555,14356,14358,14359,14338,14238,14188,14272,14401,14215,14609


In [103]:
# df_reads.reset_index(inplace=True) ##accidentally reset index twice D: whoops
df_reads.sort_values(by=['frequency'], ascending=False)

,level_0,index,fastq,read,frequency,plasmid,replicate,edit nt 1,edit nt 2,edit nt 3,edit nt 4,edit nt 5,edit nt 6,edit nt 7,edit nt 8,edit nt 9,edit nt 10,edit nt 11
4573,4573,0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,24513,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
132548,132548,0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,24513,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
84123,84123,0,msCF-2-10,AGGATGCTGCAGATCTGTTCCCTACATCACCGCCAATTGCAGATCG...,21106,pCF.91,Rep3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45990,45990,0,msCF-2-21,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,21074,pCF.89,Rep4,NaN,NaN,NaN,C,T,C,C,C,NaN,NaN,NaN
84115,84115,0,msCF-2-10,AGGATGCTGCAGATCGGAAGAGTTTACATCACCGCCAATTGCAGAT...,20089,pCF.91,Rep3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49409,49409,0,msCF-2-21,CGGCTGTCGCGGTGAATATCTCCAGATCGGAAGAGCACACGTCTGA...,1,pCF.89,Rep4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49408,49408,0,msCF-2-21,CGGCTGTCGCGTGAATATCTCCAGATCGGAAGAGCACACGTCTGAA...,1,pCF.89,Rep4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49407,49407,0,msCF-2-21,CGGCTGTCGCGTGAATATCTCCAGATCGGAAGAGCACACGTCTGAA...,1,pCF.89,Rep4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49406,49406,0,msCF-2-21,CGGCTGTGCGTGAATATCTCCAGATCGGAAGAGCACACGTCTGAAC...,1,pCF.89,Rep4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
#remove reads that do not have values (NaNs) in the specified edit sites in the above dictionary 
list_df_reads_byplasmid = []
dict_df_reads_byplasmid = dict()
for key in plasmid_to_numedits: 
    df_part = df_reads[df_reads['plasmid'] == key].dropna(subset = plasmid_to_numedits[key])#, inplace = True)
    list_df_reads_byplasmid.append(df_part)
    dict_df_reads_byplasmid[key] = df_part

df_reads_new = pd.concat(list_df_reads_byplasmid)

In [118]:
df_reads_new
df_reads_new.to_excel("df_cleanedup.xlsx")

In [150]:
df_reads_new.sort_values(by=['frequency'], ascending=False)

,level_0,index,fastq,read,frequency,plasmid,replicate,edit nt 1,edit nt 2,edit nt 3,edit nt 4,edit nt 5,edit nt 6,edit nt 7,edit nt 8,edit nt 9,edit nt 10,edit nt 11
4573,4573,0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,24513,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
132548,132548,0,msCF-2-24,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,24513,pCF.92,Rep4,C,G,C,C,T,C,C,C,C,C,C
45990,45990,0,msCF-2-21,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,21074,pCF.89,Rep4,NaN,NaN,NaN,C,T,C,C,C,NaN,NaN,NaN
21252,21252,0,msCF-2-07,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,19586,pCF.88,Rep3,NaN,NaN,NaN,NaN,T,C,C,NaN,NaN,NaN,NaN
67046,67046,0,msCF-2-22,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...,19498,pCF.90,Rep4,NaN,NaN,C,C,T,C,C,C,C,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54322,54322,0,msSBK-32-22,CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATAACGAA...,1,pCF.90,Rep1,NaN,NaN,C,C,T,C,C,C,C,NaN,NaN
54318,54318,0,msSBK-32-22,GCTGTCGCGGTTGAATATGACCAGCCAACGTCCGATATCACGAAGG...,1,pCF.90,Rep1,NaN,NaN,C,C,T,C,C,C,C,NaN,NaN
54316,54316,0,msSBK-32-22,GGATGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAAG...,1,pCF.90,Rep1,NaN,NaN,C,C,T,C,C,C,C,NaN,NaN
54314,54314,0,msSBK-32-22,GATGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAAGG...,1,pCF.90,Rep1,NaN,NaN,C,C,T,C,C,C,C,NaN,NaN


In [124]:
display(list_df_reads_byplasmid)

[       level_0  index        fastq  \
 6224      6224      0  msSBK-32-20   
 6225      6225      0  msSBK-32-20   
 6230      6230      0  msSBK-32-20   
 6232      6232      0  msSBK-32-20   
 6233      6233      0  msSBK-32-20   
 ...        ...    ...          ...   
 26766    26766      0    msCF-2-20   
 26769    26769      0    msCF-2-20   
 26770    26770      0    msCF-2-20   
 26771    26771      0    msCF-2-20   
 26773    26773      0    msCF-2-20   
 
                                                     read frequency plasmid  \
 6224   CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...      6031  pCF.88   
 6225   CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...      3633  pCF.88   
 6230   CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...        61  pCF.88   
 6232   CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...        71  pCF.88   
 6233   CGGCTGTCGCGGATGAATATGACCAGCCAACGTCCGATATCACGAA...         1  pCF.88   
 ...                                                  ..

In [164]:
#concatenating the nucleotides in the edit positions specified in all possible combinations (ex. edit nt 5 with 6 and 7, edit nt 6 with 7) 
#counting the frequencing of those concatenated pairs 
dict_edit_percent = dict()
for key in dict_df_reads_byplasmid:
    df_byplasmid = dict_df_reads_byplasmid[key]
    # display(df_byplasmid)
    # df_byplasmid.dropna(axis=1, how='all', inplace=True)
    edit_columns = [i for i in list(df_byplasmid.columns) if i[:7]=='edit nt']
    combos_columns = list(itertools.combinations(edit_columns, 2))
    for combo in combos_columns:
        df_byplasmid[combo] = df_byplasmid[combo[0]] + df_byplasmid[combo[1]]
        top = df_byplasmid.groupby(["replicate", combo])["frequency"].sum()
        bottom = df_byplasmid.groupby(["replicate"])["frequency"].sum()
        
        edit_percent = pd.DataFrame(top / bottom * 100)
        # display(edit_percent)
        if key not in dict_edit_percent:
            dict_edit_percent[key] = []
        if not edit_percent.empty:
            dict_edit_percent[key].append(edit_percent)
        else:
            # print('empty!')
            pass
    # print(combos_columns)

dict_edit_percent

{'pCF.88': [                                  frequency
  replicate (edit nt 5, edit nt 6)           
  Rep1      CC                        0.76331
            CT                      17.466705
            GC                       0.012775
            TC                      81.006675
            TT                       0.750535
  Rep2      AC                       0.014141
            AT                       0.014141
            CC                       0.820194
            CT                      17.287704
            GC                       0.035353
            TA                       0.021212
            TC                      80.888072
            TG                       0.007071
            TT                       0.912112
  Rep3      AC                       0.005899
            CC                       1.262462
            CT                      22.792166
            GC                       0.008849
            TC                       74.49413
            TT          

In [253]:
#making a dictionary of all the double edited combinations (ee) 
edited_combos = dict()
edited_nucleotide_df = samples[[i for i in list(samples.columns) if i[0:5]=='Ed nt']].dropna()
for in list(itertools.combinations(list(samples[[i for i in list(samples.columns) if i[0:5]=='Ed nt']].columns), 2)):
    edited_combos[pair] = edited_nucleotide_df[pair[0]].values[0] + edited_nucleotide_df[pair[1]].values[0]
    
edited_combos

SyntaxError: invalid syntax (440189757.py, line 4)

In [248]:
edited_nucleotide_df 

,Ed nt 1,Ed nt 2,Ed nt 3,Ed nt 4,Ed nt 5,Ed nt 6,Ed nt 7,Ed nt 8,Ed nt 9,Ed nt 10,Ed nt 11
4,T,A,T,T,C,T,T,T,T,T,T


In [219]:
#using that dictionary of double edited combination to map onto the df of results 
#keep only the frequencies of the double edited sites (ee) 
#average the replicates 
dict_edit_percent_ee = dict()
dict_edit_percent_ee_mean = dict()
for plasmid in dict_edit_percent:
    if plasmid not in dict_edit_percent_ee:
        dict_edit_percent_ee[plasmid] = dict()
    if plasmid not in dict_edit_percent_ee_mean:
        dict_edit_percent_ee_mean[plasmid] = dict()
        
    for pair in dict_edit_percent[plasmid]:
        matching_index = tuple([ 'Ed' + i[4:] for i in list(pair.index.names[1])])
        # print(matching_index)
        
        # if pair[(pair.index.get_level_values(tuple(pair.index.names[1]))==edited_combos[matching_index])].empty:
        #     print(tuple(pair.index.names[1]))
        #     print(edited_combos[matching_index])
        #     print(pair)
        #     print(pair[(pair.index.get_level_values(tuple(pair.index.names[1]))==edited_combos[matching_index])])
        
        try:
            dict_edit_percent_ee_mean[plasmid][tuple(pair.index.names[1])] = pair[(pair.index.get_level_values(tuple(pair.index.names[1]))==edited_combos[matching_index])].mean().values[0]
            # print(pair[(pair.index.get_level_values(tuple(pair.index.names[1]))==edited_combos[matching_index])].mean().values[0])
        except:
            dict_edit_percent_ee_mean[plasmid][tuple(pair.index.names[1])] = 0
            # print(0)
        
        dict_edit_percent_ee[plasmid][tuple(pair.index.names[1])] = pair[(pair.index.get_level_values(tuple(pair.index.names[1]))==edited_combos[matching_index])]

In [220]:
dict_edit_percent_ee_mean

{'pCF.88': {('edit nt 5', 'edit nt 6'): 20.376580862927092,
  ('edit nt 5', 'edit nt 7'): 19.828066522245255,
  ('edit nt 6', 'edit nt 7'): 20.859249673504138},
 'pCF.89': {('edit nt 4', 'edit nt 5'): 0.023286423189944453,
  ('edit nt 4', 'edit nt 6'): nan,
  ('edit nt 4', 'edit nt 7'): nan,
  ('edit nt 4', 'edit nt 8'): nan,
  ('edit nt 5', 'edit nt 6'): nan,
  ('edit nt 5', 'edit nt 7'): nan,
  ('edit nt 5', 'edit nt 8'): nan,
  ('edit nt 6', 'edit nt 7'): nan,
  ('edit nt 6', 'edit nt 8'): nan,
  ('edit nt 7', 'edit nt 8'): nan},
 'pCF.90': {('edit nt 3', 'edit nt 4'): 5.5720865753799025,
  ('edit nt 3', 'edit nt 5'): 5.099578640466136,
  ('edit nt 3', 'edit nt 6'): 4.785114001110522,
  ('edit nt 3', 'edit nt 7'): 4.572395242671736,
  ('edit nt 3', 'edit nt 8'): 4.018180186138991,
  ('edit nt 3', 'edit nt 9'): 3.151578796103758,
  ('edit nt 4', 'edit nt 5'): 6.790402003548711,
  ('edit nt 4', 'edit nt 6'): 6.40452349906008,
  ('edit nt 4', 'edit nt 7'): 6.09781352308812,
  ('edit nt

In [239]:
#organizing dictionary of edits into a format suitable for a heatmap 
dict_df_heatmap = dict()
for plasmid in dict_edit_percent_ee_mean:
    if plasmid not in dict_df_heatmap:
        potato = list(set([j for i in list(dict_edit_percent_ee_mean[plasmid].keys()) for j in i]))
        # print(potato)
        dict_df_heatmap[plasmid] = pd.DataFrame(columns=potato, index=potato)
    # print(dict_edit_percent_ee_mean[plasmid])
    for pair in dict_edit_percent_ee_mean[plasmid]:
        dict_df_heatmap[plasmid].loc[pair[0], pair[1]] = dict_edit_percent_ee_mean[plasmid][pair]
        dict_df_heatmap[plasmid].loc[pair[1], pair[0]] = dict_edit_percent_ee_mean[plasmid][pair]
        # print(dict_df_heatmap[plasmid].loc[pair[0], pair[1]])

In [240]:
dict_df_heatmap

{'pCF.88':            edit nt 5  edit nt 6  edit nt 7
 edit nt 5        NaN  20.376581  19.828067
 edit nt 6  20.376581        NaN   20.85925
 edit nt 7  19.828067   20.85925        NaN,
 'pCF.89':           edit nt 4 edit nt 5 edit nt 8 edit nt 6 edit nt 7
 edit nt 4       NaN  0.023286       NaN       NaN       NaN
 edit nt 5  0.023286       NaN       NaN       NaN       NaN
 edit nt 8       NaN       NaN       NaN       NaN       NaN
 edit nt 6       NaN       NaN       NaN       NaN       NaN
 edit nt 7       NaN       NaN       NaN       NaN       NaN,
 'pCF.90':           edit nt 4 edit nt 5 edit nt 9 edit nt 3 edit nt 8 edit nt 6  \
 edit nt 4       NaN  6.790402  4.293997  5.572087   5.42585  6.404523   
 edit nt 5  6.790402       NaN  5.201341  5.099579  6.530956  7.745636   
 edit nt 9  4.293997  5.201341       NaN  3.151579  7.112324  5.746553   
 edit nt 3  5.572087  5.099579  3.151579       NaN   4.01818  4.785114   
 edit nt 8   5.42585  6.530956  7.112324   4.01818      

In [247]:
for plasmid in dict_df_heatmap: 
    dict_df_heatmap[plasmid].to_csv(f"{plasmid}_heat_map.csv")